# Notebook to select host galaxies and submit candidates to TNS

In [1]:
import sys, os
import pandas as pd
import numpy as np
import ipyaladin as ipyal
from ipywidgets import Layout, Box, widgets
import json
import time
import pickle

In [2]:
from astroquery.ned import Ned
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.table import Table, Column

In [3]:
sys.path.append("./lib")
from alerce_tns import *

In [4]:
from delight.delight import Delight

In [5]:
# continuously load modules (for easier debugging)
%load_ext autoreload
%autoreload 2

# Start API

In [12]:
alerce = alerce_tns()

# Load recent candidates

**WARNING: CHECK THE DATE**

In [13]:
refstring = "20220723"
candidates_dir = "candidates"
candidates = open("%s/%s.csv" % (candidates_dir, refstring), encoding='utf-8-sig').read().splitlines()
if "" in candidates:
    candidates.remove("") 
candidates = np.sort(np.unique(np.array([cand.strip() for cand in candidates])))
print(candidates, len(candidates))

['ZTF18aanabqp' 'ZTF22aatpwfw' 'ZTF22aatxslc' 'ZTF22aatylyb'
 'ZTF22aatymow' 'ZTF22aaudjgc' 'ZTF22aaugvnu' 'ZTF22aauhudo'
 'ZTF22aaujkdb' 'ZTF22aaunsga' 'ZTF22aautnrr' 'ZTF22aautntp'
 'ZTF22aautpdb' 'ZTF22aautrcf' 'ZTF22aauuvby' 'ZTF22aauuxul'
 'ZTF22aauvseh' 'ZTF22aauwylz' 'ZTF22aauxltc' 'ZTF22aauxsyt'
 'ZTF22aavcccu' 'ZTF22aavclih' 'ZTF22aavcrgy' 'ZTF22aavcvtr'
 'ZTF22aavcyir' 'ZTF22aavczfb' 'ZTF22aavdcsu' 'ZTF22aavdrbx'
 'ZTF22aavdrdz' 'ZTF22aavdscs' 'ZTF22aavdubl' 'ZTF22aavdzpf'
 'ZTF22aavebae' 'ZTF22aavecme' 'ZTF22aaveghe' 'ZTF22aavfgyf'
 'ZTF22aavgzjc' 'ZTF22aavicyh' 'ZTF22aavjwht' 'ZTF22aavklii'] 40


In [14]:
stats = alerce.query_objects(oid=candidates, format='pandas', page_size=len(candidates))
delight_datadir = "delight_data"
dclient = Delight(delight_datadir, candidates, stats.meanra.to_numpy(), stats.meandec.to_numpy())
dclient.download()
dclient.get_pix_coords()
dclient.compute_multiresolution(nlevels=5, domask=False, doobject=True, doplot=False)
model_dir = "/home/fforster/Dropbox/Work/DELIGHT/data/model"
model_name = "DELIGHT_v1.h5"
dclient.load_model(model_dir, model_name)
dclient.preprocess()
dclient.predict()

Loading WCS information


In [15]:
#for oid in candidates:
#    dclient.plot_host(oid)

# Select host galaxies using aladin widget

## Do automatic selection

In [16]:
bestguess = dclient.df[["ra_delight", "dec_delight"]].rename({"ra_delight": "ra", "dec_delight": "dec"}, axis=1)
alerce.bestguess = bestguess

In [17]:
alerce.select_hosts(candidates, refstring, ned=True, simbad=True, catsHTM=False, vizier=False)

Loading and skipping already saved hosts...
hosts/20220723_hosts.csv
Cannot load galaxy information, creating new information.


Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

,host_name,host_ra,host_dec,host_offset,host_offset_bestguess,host_source,host_redshift_spec,host_redshift,host_redshift_error,host_redshift_type
oid,,,,,,,,,,
ZTF18aanabqp,SDSS J152003.79+451811.3,230.015804,45.303156,1.230081,0.352793,SDSSDR16,True,0.098949,0.00001,specz


39 candidates remaining


Box(children=(Aladin(fov=0.025, layout=Layout(width='70%'), options=['allow_full_zoomout', 'coo_frame', 'fov',…

...

/home/fforster/anaconda3/lib/python3.8/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '5:10:48.1422 +28:40:25.0101': No astronomical object found :
  warnings.warn("Warning: The script line number %i raised "


,host_name,host_ra,host_dec,host_offset,host_offset_bestguess,host_source,host_redshift_spec,host_redshift,host_redshift_error,host_redshift_type
oid,,,,,,,,,,
ZTF22aavklii,WISEA J051048.14+284024.4,77.70063,28.67344,0.748188,0.637173,NED,False,0.022256,NaN,PHOT


0 candidates remaining


Summary of host galaxies:
Saving hosts to hosts/20220723_hosts.csv


,host_name,host_ra,host_dec,host_offset,host_offset_bestguess,host_source,host_redshift_spec,host_redshift,host_redshift_error,host_redshift_type
oid,,,,,,,,,,
ZTF22aavklii,WISEA J051048.14+284024.4,77.70063,28.67344,0.748188,0.637173,NED,False,0.022256,NaN,PHOT
ZTF22aavjwht,SDSS J003701.95+184939.0,9.258119,18.827518,0.448959,0.279929,SDSSDR16,False,0.150257,0.035163,photoz
ZTF22aavicyh,SDSS J005310.39-070334.1,13.293302,-7.059494,0.370979,0.295969,SDSSDR16,False,0.143345,0.070323,photoz
ZTF22aavgzjc,WISEA J002816.83+510706.3,7.07014,51.11843,0.22963,0.146808,NED,False,NaN,NaN,NaN
ZTF22aavfgyf,2MASX J22102557-0917296,332.60670166666665,-9.2914775,5.095737,0.385926,Simbad,True,0.07012,0.00001,C
ZTF22aaveghe,SDSS J211616.28-022232.8,319.067849,-2.375778,1.400294,0.576217,SDSSDR16,False,0.117027,0.013094,photoz
ZTF22aavecme,SDSS J232137.80+301559.5,350.407492,30.266537,2.732855,0.530824,SDSSDR16,False,0.1266,0.029964,photoz
ZTF22aavebae,SDSS J000034.44+273640.5,0.143508,27.611273,1.219457,0.47243,SDSSDR16,False,0.167596,0.037761,photoz
ZTF22aavdzpf,WISEA J224634.04+394557.4,341.64188,39.76654,9.747363,1.888651,NED,False,NaN,NaN,NaN


Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to hosts/20220723_hosts.csv
Saving hosts to 

## Do manual selections for errors

Errors could happen because of an error in DELIGHT or because the association favoured a more distant host with redshift.

In [20]:
errors = []#"ZTF22aaujkdb", "ZTF22aaveghe"]
# confirm if this was a problem with DELIGHT
for oid in errors:
    dclient.plot_host(oid)
# do manual selection
if errors != []:
    alerce.select_hosts(errors, refstring, ned=True, simbad=True, catsHTM=False, vizier=False, dobestguess=False)

## Manual modifications

In [158]:
#oid = "ZTF21acbcfmh" #ZTF21abidhcz"
##oid = 'ZTF20aclzygy'#ZTF20acgigfo'
##oid = "ZTF20aaqrmxg" #"ZTF20aakdppm" #ZTF19acxrihd"
#oid = 'ZTF22aaajgss' #ZTF21aaryakx' #ZTF20acvbneg'
#alerce.candidate_hosts.host_name[oid] = "M31"
#alerce.candidate_hosts.host_ra[oid] = ""
#alerce.candidate_hosts.host_dec[oid] = ""
#alerce.candidate_hosts.host_offset[oid] = ""
#alerce.candidate_hosts.host_source[oid] = "CDS"
#alerce.candidate_hosts.host_redshift_spec[oid] = True
#alerce.candidate_hosts.host_redshift[oid] = = "0.001341"
#alerce.candidate_hosts.host_redshift_error[oid] = "0.000040"
#alerce.candidate_hosts.host_redshift_type[oid] = "specz"
# #"nan" #"0.041" #nan"
##for i in list(alerce.candidate_hosts):
##    alerce.candidate_hosts.loc['ZTF21aaryakx'][i] = "NULL"

## Save hosts

In [159]:
alerce.candidate_hosts.to_csv("hosts/%s_hosts.csv" % refstring)
alerce.candidate_hosts.head(100)

,host_name,host_ra,host_dec,host_offset,host_source,host_redshift_spec,host_redshift,host_redshift_error,host_redshift_type
oid,,,,,,,,,
ZTF22aavklii,WISEA J051048.14+284024.4,77.700630,28.673440,0.748188,NED,False,0.022256,NULL,PHOT
ZTF22aavjwht,SDSS J003701.95+184939.0,9.258119,18.827518,0.448959,SDSSDR16,False,0.150257,0.035163,photoz
ZTF22aavicyh,SDSS J005310.39-070334.1,13.293302,-7.059494,0.370979,SDSSDR16,False,0.143345,0.070323,photoz
ZTF22aavgzjc,WISEA J002816.83+510706.3,7.070140,51.118430,0.229630,NED,NULL,NULL,NULL,NULL
ZTF22aavfgyf,2MASX J22102557-0917296,332.606720,-9.291480,5.157590,NED,True,0.070118,NULL,SPEC
ZTF22aaveghe,SDSS J211616.28-022232.8,319.067849,-2.375778,1.400294,SDSSDR16,False,0.117027,0.013094,photoz
ZTF22aavecme,SDSS J232137.80+301559.5,350.407492,30.266537,2.732855,SDSSDR16,False,0.1266,0.029964,photoz
ZTF22aavebae,SDSS J000034.44+273640.5,0.143508,27.611273,1.219457,SDSSDR16,False,0.167596,0.037761,photoz
ZTF22aavdzpf,WISEA J224634.04+394557.4,341.641880,39.766540,9.747363,NED,NULL,NULL,NULL,NULL


## Show nearest sources (z < 0.01)

In [160]:
zth = 0.1
nearest = pd.to_numeric(alerce.candidate_hosts.loc[alerce.candidate_hosts.host_redshift != "NULL"].host_redshift).sort_values()
nearest = nearest.loc[nearest < zth]
for i in ["https://alerce.online/object/%s %s" % (i, alerce.candidate_hosts.host_redshift.loc[i]) for i in nearest.index]:
    print(i)

https://alerce.online/object/ZTF22aavklii 0.022256
https://alerce.online/object/ZTF22aaudjgc 0.02719698
https://alerce.online/object/ZTF22aatpwfw 0.03109159
https://alerce.online/object/ZTF22aavfgyf 0.070118
https://alerce.online/object/ZTF22aatylyb 0.080097
https://alerce.online/object/ZTF22aauuxul 0.084925
https://alerce.online/object/ZTF22aatxslc 0.0873167
https://alerce.online/object/ZTF18aanabqp 0.09894894


# Prepare TNS report

## Sandbox or real

In [161]:
sandbox = False #False # True: test in sandbox, False: send to real website

## Get API key and url

In [162]:
if sandbox:
    print("Getting API sandbox key...")
    api_key = open("API_sandbox.key", "r").read()
    api_key = open("API.key", "r").read() # sandbox key appears not to be working
else:
    print("Getting API key...")
    api_key = open("API.key", "r").read()
    
if sandbox:
    url_tns_api = "https://sandbox.wis-tns.org/api" #"https://www.wis-tns.org/" #"https://sandbox-tns.weizmann.ac.il/api"
else:
    url_tns_api = "https://www.wis-tns.org/api" #"https://wis-tns.weizmann.ac.il/api" #"https://www.wis-tns.org/" #"https://wis-tns.weizmann.ac.il/api"
    
if not sandbox:
    print("WARNING: submitting to real (not sandbox) TNS website")

Getting API key...


## Confirm reporter names

This is also used for Skyportal

In [163]:
reporter = "F. Forster, F.E. Bauer, G. Pignata, A. Munoz-Arancibia, A. Mourao, R. Dastidar, L. Hernandez-Garcia, L. Galbany, E. Camacho, J. Silva-Farfan, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker"
#F. Forster, F.E. Bauer, A. Munoz-Arancibia, G. Pignata, L. Hernandez-Garcia, L. Galbany, E. Camacho, J. Silva-Farfan, A. Mourao, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker"

## Build the report

In [164]:
# first author: the person submitting, 2nd author: those that vetoed, then everyone else
tns_report = {"at_report": {}}
counter = 0
for oid in candidates[::-1]: # invert to report old candidates first
    #if oid in ["ZTF20aauqmwf"]:#["ZTF19acgjpfd", "ZTF18acszatf", "ZTF18acrkmnm"]:
    #    continue
    #if oid in ["ZTF20acqnfkh ", "ZTF20acneabh"]:#ZTF20acplctd":#aamtgd": #ZTF19acymtgr": #ZTF19acnqqhl":#in ["ZTF19acmdwjy", "ZTF19acmdwjz"]:
    #    continue
    #if oid == "ZTF20ablnjwh": #ZTF20abkmxrm":#ZTF20abgfgne":
    #    continue
    if oid in ["ZTF22aabrskg"]: #ZTF21abgvidd"]: #ZTF21aawgsfp"]:#ZTF21aalcrjh"]: #ZTF21aaanwxo"]:#, ZTF20adadixi"]:# ZTF20acsggey"]:#, "ZTF20acmevcc", "ZTF20acmctng", "ZTF20acmdvzk"]: #== "ZTF20acgknep":#"ZTF20acdcdqq": #ZTF20acaehey": #ZTF20abyjoxb":#"ZTF20abqjddh":#ZTF20abkxqzv":#ZTF20abgbfyk":
        continue

    print("\n", counter, oid)
    sn_report = alerce.do_TNS_report(api_key, oid, reporter, verbose=False, test=False)
    if sn_report:# or oid == "ZTF20abyptpc":
        tns_report["at_report"]["%i" % counter] = sn_report
        counter += 1



 0 ZTF22aavklii
http://alerce.online/object/ZTF22aavklii


SN candidate classified using ALeRCE's stamp classifier (Carrasco-Davis et al. 2021) and the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF22aavklii 

 1 ZTF22aavjwht
http://alerce.online/object/ZTF22aavjwht


SN candidate classified using ALeRCE's stamp classifier (Carrasco-Davis et al. 2021) and the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF22aavjwht 

 2 ZTF22aavicyh
http://alerce.online/object/ZTF22aavicyh


SN candidate classified using ALeRCE's stamp classifier (Carrasco-Davis et al. 2021) and the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF22aavicyh 

 3 ZTF22aavgzjc
http://alerce.online/object/ZTF22aavgzjc


SN candidate classified using ALeRCE's stamp classifier (Carrasco-Davis et al. 2021) and the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF22aavgzjc 

 4 ZTF22aavfgyf
http://alerce.online/object/ZTF22aavfgyf


...

SN candidate classified using ALeRCE's stamp classifier (Carrasco-Davis et al. 2021) and the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF22aatymow 

 34 ZTF22aatylyb
http://alerce.online/object/ZTF22aatylyb


SN candidate classified using ALeRCE's stamp classifier (Carrasco-Davis et al. 2021) and the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF22aatylyb 
Sleep for 41 sec

 35 ZTF22aatxslc
http://alerce.online/object/ZTF22aatxslc


SN candidate classified using ALeRCE's stamp classifier (Carrasco-Davis et al. 2021) and the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF22aatxslc 
Astronomical transient is known: [{'objname': '2022pfw', 'prefix': 'AT', 'objid': 112350}] 

Reporting info: {'discoverer': ['ZTF'], 'objname': ['2022pfw'], 'reporter': ['SGLF'], 'internal_names': ['ZTF22aatxslc', 'PS22ggs']}
Object was reported using the same ZTF internal name, do not report.

 35 ZTF22aatpwfw
http://alerce.online/object/ZTF22aatpwfw


Early SN candidate (g-rise > 0.14+-0.01 mag/day, r-rise > 0.10+-0.01 mag/day) classified using ALeRCE's stamp classifier (Carrasco-Davis et al. 2021) and the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF22aatpwfw 
Astronomical transient is known: [{'objname': '2022paf', 'prefix': 'AT', 'objid': 112197}] 

Reporting info: {'discoverer': ['ZTF'], 'objname': ['2022paf'], 'reporter': ['ZTF'], 'internal_names': ['ZTF22aatpwfw']}
Object was reported using the same ZTF internal name, do not report.

 35 ZTF18aanabqp
http://alerce.online/object/ZTF18aanabqp


Early SN candidate (r-rise > 0.16+-0.03 mag/day) classified using ALeRCE's stamp classifier (Carrasco-Davis et al. 2021) and the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF18aanabqp 


## Manual modifications

In [165]:
#report['at_report']['0']['remarks'] = "Fast rising transient (g-rise > 1.48 mag/day)"#report['at_report']['3']['remarks'] = 'Nova candidate in M31 (g-rise > 0.16+-0.01 mag/day, r-rise > 0.24+-0.01 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF19acxrihd '

In [166]:
# note that the space at the end is necessary
#report['at_report']['3']['remarks'] = 'Nova candidate in M31 (r-rise > 0.43+-0.01 mag/day) classified by ALeRCE using the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF20acgigfo '

## Display report

In [167]:
print(json.dumps(tns_report, indent=4, sort_keys=True))

{
    "at_report": {
        "0": {
            "at_type": "1",
            "dec": {
                "error": "0.085",
                "units": "arcsec",
                "value": "28.67364345"
            },
            "discovery_data_source_id": "48",
            "discovery_datetime": "2022-07-23 11:38:01.000",
            "host_name": "WISEA J051048.14+284024.4",
            "internal_name": "ZTF22aavklii",
            "non_detection": {
                "comments": "Data provided by ZTF",
                "exptime": "30",
                "filter_value": "111",
                "flux_units": "1",
                "instrument_value": "196",
                "limiting_flux": "18.4507",
                "obsdate": "2022-07-22 11:49:29.997",
                "observer": "Robot"
            },
            "photometry": {
                "photometry_group": {
                    "0": {
                        "comments": "Data provided by ZTF",
                        "exptime": "30",
          

## Look for fast risers

In [168]:
for i in tns_report["at_report"]:
    if tns_report["at_report"][i]["remarks"][:5] == "Early":
        try:
            rise = float(re.findall("rise\s>\s(.*?)\+-.*?\smag", tns_report["at_report"][i]["remarks"])[0])
            if rise >= 0.25:
                print(tns_report["at_report"][i]["remarks"])
        except:
            rise = float(re.findall("rise\s>\s(.*?)\s", tns_report["at_report"][i]["remarks"])[0])
            if rise >= 0.25:
                print(tns_report["at_report"][i]["remarks"])

Early SN candidate (g-rise > 0.31+-0.07 mag/day) classified using ALeRCE's stamp classifier (Carrasco-Davis et al. 2021) and the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF22aautpdb 
Early SN candidate (r-rise > 1.00 mag/day) classified using ALeRCE's stamp classifier (Carrasco-Davis et al. 2021) and the public ZTF stream. Discovery image and light curve in http://alerce.online/object/ZTF22aauhudo 


## Save report

In [169]:
json_report = '%s/%s.json' % (candidates_dir, refstring)
with open(json_report, 'w') as outfile:
    json.dump(tns_report, outfile)

## Send to TNS

**WARNING: key is used to send candidates!**

In [170]:
json_report

'candidates/20220723.json'

In [171]:
r = alerce.send_json_report(api_key, url_tns_api, json_report)

## Show output

In [172]:
r.content

b'{"id_code":200,"id_message":"OK","data":{"report_id":110943,"recieved_data":{"at_report":{"0":{"ra":{"value":77.700678375,"error":0.085,"units":"arcsec"},"dec":{"value":28.67364345,"error":0.085,"units":"arcsec"},"reporting_group_id":74,"discovery_data_source_id":48,"reporter":"F. Forster, F.E. Bauer, G. Pignata, A. Munoz-Arancibia, A. Mourao, R. Dastidar, L. Hernandez-Garcia, L. Galbany, E. Camacho, J. Silva-Farfan, J. Arredondo, G. Cabrera-Vives, R. Carrasco-Davis, P.A. Estevez, P. Huijse, E. Reyes, I. Reyes, P. Sanchez-Saez, C. Valenzuela, E. Castillo, D. Ruz-Mieres, D. Rodriguez-Mancini, M. Catelan, S. Eyheramendy, M.J. Graham on behalf of the ALeRCE broker","discovery_datetime":"2022-07-23 11:38:01.000","at_type":1,"internal_name":"ZTF22aavklii","remarks":"SN candidate classified using ALeRCE\'s stamp classifier (Carrasco-Davis et al. 2021) and the public ZTF stream. Discovery image and light curve in http:\\/\\/alerce.online\\/object\\/ZTF22aavklii ","non_detection":{"obsdate":

## Show feedback

In [173]:
# function for getting reply from report                             
def reply(url, report_id):                                           
    try:                                                               
        # url for getting report reply                                   
        reply_url=url+'/bulk-report-reply'                               
        # construct list of (key,value) pairs                                
        reply_data=[('api_key',(None, api_key)),                         
                    ('report_id',(None,report_id))]                      
        # send report ID using request module                            
        response=requests.post(reply_url, files=reply_data, headers=alerce.tns_headers)              
        # return response                                                
        return response                                                  
    except Exception as e:                                             
        return [None,'Error message : \n'+str(e)]  

In [27]:
try:
    feedback = reply(url_tns_api, r.json()["data"]["report_id"]).json()["data"]["feedback"]
    print(feedback)
except:
    print("Report not available, probably being processed. Please try again...")

{'at_report': [{'100': {'message': 'Transient object was inserted.', 'objname': '2022pji', 'message_id': 100}, 'at_rep': {'103': {'message': 'Submitted', 'message_id': 103}}}]}


## Check the following link and confirm that the latest SNe are from the reported day

https://www.wis-tns.org/search?&discovered_period_value=1&discovered_period_units=days&reporting_groupid[]=74&discovery_data_source_id=48&num_page=50&format=html&edit[type]=&edit[objname]=&edit[id]=&display[redshift]=1&display[hostname]=1&display[host_redshift]=1&display[source_group_name]=1&display[classifying_source_group_name]=1&display[discovering_instrument_name]=0&display[classifing_instrument_name]=0&display[programs_name]=0&display[internal_name]=1&display[isTNS_AT]=0&display[public]=1&display[end_pop_period]=0&display[spectra_count]=1&display[discoverymag]=1&display[discmagfilter]=1&display[discoverydate]=1&display[discoverer]=1&display[remarks]=0&display[sources]=0&display[bibcode]=0&display[ext_catalogs]=0&sort=desc&order=discoverydate

# Prepare SkyPortal report

In [15]:
alerce.candidate_hosts = pd.read_csv("hosts/%s_hosts.csv" % refstring)
alerce.candidate_hosts.set_index("oid", inplace=True)
alerce.candidate_hosts.fillna("NULL", inplace=True)
alerce.candidate_hosts

,host_name,host_ra,host_dec,host_offset,host_source,host_redshift_spec,host_redshift,host_redshift_error,host_redshift_type
oid,,,,,,,,,
ZTF21acmqvbd,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
ZTF21acmoazo,SDSS J095114.37+330955.2,147.809891,33.165342,2.588265,SDSSDR16,False,0.134061,0.037996,photoz
ZTF21acmnaqg,WISEA J080031.69-062939.8,120.13169,-6.49463,7.002093,NED,NULL,NULL,NULL,NULL
ZTF21acmjdkw,GALEXASC J065243.72+541801.4,103.18218,54.30041,0.489825,NED,NULL,NULL,NULL,NULL
ZTF21acmiozw,WISEA J001315.44+445453.0,3.31425,44.91467,9.66068,NED,False,0.108361,NULL,PHOT
...,...,...,...,...,...,...,...,...,...
ZTF21aciwbbr,SDSS J224030.80-005111.5,340.128353,-0.853216,7.215836,SDSSDR16,True,0.088589,0.000008,specz
ZTF21aciuqse,SDSS J211437.31+100724.6,318.655445,10.123511,0.824313,SDSSDR16,False,0.143854,0.028398,photoz
ZTF21acisvbk,SDSS J011411.55+075415.0,18.548134,7.904168,1.04024,SDSSDR16,False,0.07159,0.021361,photoz


## Load credentials

In [26]:
skyportal_url = "http://desi2.lbl.gov:5000/api"
credentials_file = "credentials_skyportal.json"
with open(credentials_file) as jsonfile:
    params = json.load(jsonfile)
token = params["token"]

## Build the report

In [27]:
skyportal_report = []
counter = 0
for oid in candidates[::-1]: # invert to report old candidates first
    if oid in ["ZTF21abkvttz"]: #ZTF21abhpdgd", "ZTF21abgtlir"]:
        continue
    print("\n", counter, oid)
    sn_report = alerce.do_skyportal_report(skyportal_url, token, oid, reporter, verbose=False, test=False)
    if sn_report:# or oid == "ZTF20abyptpc":
        skyportal_report.append(sn_report)
        counter += 1


 0 ZTF21acistjw


KeyError: 'ALERCE'

## Save report

In [29]:
with open('%s_skyportal.pickle' % refstring, 'wb') as handle:
    pickle.dump(skyportal_report, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('%s_skyportal.pickle' % refstring, 'rb') as handle:
    skyportal_report = pickle.load(handle)
    
display(skyportal_report)

[]

## Submit to SkyPortal

In [33]:
for report in skyportal_report:
    print(report["candidates"]["id"])
    alerce.send_skyportal_report(skyportal_url, token, report)

ZTF21aciiklb

Reporting candidate
HTTP code: 200, OK
Candidate sent


{'status': 'success',
 'data': {'ids': [143374]},
 'version': '0.9.dev0+git%aI.66d9fd0'}


Sending annotation

HTTP code: 400, Bad Request


{'status': 'error',
 'message': 'HTTP 405: Method Not Allowed',
 'data': {},
 'version': '0.9.dev0+git%aI.66d9fd0'}

ZTF21acihxwk

Reporting candidate
HTTP code: 200, OK
Candidate sent


...

{'status': 'success',
 'data': {'ids': [143395]},
 'version': '0.9.dev0+git%aI.66d9fd0'}


Sending annotation

HTTP code: 400, Bad Request


{'status': 'error',
 'message': 'HTTP 405: Method Not Allowed',
 'data': {},
 'version': '0.9.dev0+git%aI.66d9fd0'}

ZTF18aawitry
ZTF18aawitry is in skyportal

Sending annotation

HTTP code: 400, Bad Request


{'status': 'error',
 'message': 'HTTP 405: Method Not Allowed',
 'data': {},
 'version': '0.9.dev0+git%aI.66d9fd0'}

## Check SkyPortal

http://desi2.lbl.gov:5000/candidates